In [ ]:
import math


##-------------------
##FUNCIONES AUXILIARES
##busca todas las posibles intersecciones dentro del estado de la beach line y las adiciona a la lista de eventos,
def search_inter():
  global events
  aux_l=status['beach']
  circle_ev=[ev for ev in events if 'y' in ev]

  for i in range(len(aux_l)-2):
    if aux_l[i:i+3] not in [ev['ids'] for ev in circle_ev]:
      pass
    #verifica centro de circulos si tres existen 3 arcos consecutivos distintos en el estado
    ##si existen busca cual es la interseccion de estos arcos, si el punto inferior del circulo se encuentra bajo la linea de barrido
    ##adiciona un circle event a la linea de eventos
        #si la parte inferior del circulo se encuentra por debajo de la linea de barrido insertarlo en la fila de eventos

In [ ]:
#da la coordenada en y para un punto x con la parapola generada desde el punto p
def parabola(x,p,l):
  return (p[0]**2 - l**2 - 2 * p[0] * x + x**2 + p[1]**2) / (2 * (p[1] - l))

In [ ]:
#dados los indices de dos puntos adiciona el bisector a los los vertices del diagrama de Voronoi
def add_bisect(i1,i2):
  p1x,p1y=ind_p[i1]
  p2x,p2y=ind_p[i2]
  vor_vertices[(i1,i2)]=((p1x+p2x)/2,(p1y+p2y)/2)

In [ ]:
##da el id del arco en el beach line directamente sobre un nuevo punto p
def id_arc_above(pto):
  ind_min=-1
  minn=10000
  l=status['y']
  for p in status['beach']:
    val=parabola(pto[0],ind_p[p],l)
    if val < minn:
      ind_min=p
      minn=val
  return ind_min

In [ ]:
#procesa los eventos
#finds the centr of the circle defined by three points, caveat, no two points can have the same y coordinate
def bisector_2p(p1, p2):
    p1x, p1y = p1
    p2x, p2y = p2
    m1 = ((p2x - p1x)/(p1y - p2y) )
    b1 = (p1y + p2y) / 2 - m1 * (p1x + p2x) / 2
    return (m1, b1)

In [ ]:
def int_2lines(m1, b1, m2, b2):
    x = (b2 - b1) / (m1 - m2)
    y = m1 * x + b1
    return (x, y)

def centr_circle(p1, p2, p3):
    ##cambiar en esta parte usando las funciones auxiliares para que asigne los valores adecuados al radio y al punto (pt)
    return pt,rad

In [ ]:
#adiciona el circle event manteniendo el orden
#implementar una manera mas eficiente de adicionar en la lista de eventos
def insert_ev(i1,i2,i3,ctr,rad):
  global events
  new_ev={'ids':[i1,i2,i3],'coord':ctr, 'y':ctr[1]-rad}
  if len(events)==0:
     events=[new_ev]
  else:
    y_top=events[-1]['y'] if 'y' in events[-1] else events[-1][1]
    x_top=events[-1]['coord'][0] if 'coord' in events[-1] else events[-1][0]
    if y_top <ctr[1]-rad or (y_top==ctr[1]-rad and x_top>ctr[0]):
        events.append(new_ev)
    else:
        for i in range(len(events)):
            event=events[i]
            event_y = event['y'] if 'y' in event else event[1]
            event_x = event['coord'][0] if 'coord' in event else event[0]
            if event_y>ctr[1]-rad or (event_y==ctr[1]-rad and event_x<ctr[0]):
                events=events[:i] + [new_ev]+ events[i:]

In [ ]:
points=[(3, 6), (10, 5), (4, 3), (8, 9), (9, 3.2), (1, 8), (4, 1)]
## En el estado se mantiene el actual orden de los arcos
##
#funcion que da el valor en y  correspondiente a x en la parabola dada por la linea a altura l
# punto p
def elimina_arcos_fuera(y):
   global bd_max_x
   global bd_min_x
   while len(status['beach']) >1 and parabola(bd_min_x ,ind_p[status['beach'][0]],y) > parabola(bd_min_x ,ind_p[status['beach'][1]],y):
      status['beach']=status['beach'][1:]
   while len(status['beach']) >1 and parabola(bd_max_x ,ind_p[status['beach'][-1]],y) > parabola(bd_max_x ,ind_p[status['beach'][-2]],y):
      status['beach']=status['beach'][:-1]

In [ ]:
#define los bordes en que esta el diagrama
bd_max_x=max([p[0] for p in points])
bd_min_x=min([p[0] for p in points])
bd_max_y=max([p[1] for p in points])
bd_min_y=min([p[1] for p in points])
#en el estado se guarda en beach el orden de los segmentos de izquierda a derecha
#en el beach line, la actual altura y de la linea de barrido
status={'beach':[]}
#diccionario que como llave tiene los indices de los nodos involucrados y como valor las coordenadas
vor_vertices={}
events=sorted(points, key=lambda x: (x[1],-x[0]))
#shallow copy sirve como diccionario de indice -> punto, indice segun orden como eventos
ind_p=events[:]


In [ ]:

while len(events) >0:
  to_process=events.pop()

  #caso en que el evento a procesar sea un nuevo nodo
  if to_process in ind_p:
    status['y']=to_process[1]
    elimina_arcos_fuera(status['y'])
    print('y cambia a ',status['y'],' con ', to_process)
    #indice del arco sobre el nodo
    ind=id_arc_above(to_process)
    i_2proc=ind_p.index(to_process)
    if ind==-1:
      status['beach']=[i_2proc]
    else:
      aux_i=status['beach'].index(ind)
      add_bisect(ind,i_2proc)
      status['beach']=status['beach'][:aux_i +1]+[i_2proc]+status['beach'][aux_i:]
      print(' status cam ',[ind_p[i] for i in status['beach']])
      search_inter()
  #caso en que el evento a procesar sea un circle event
  else:
    status['y']=to_process['y']
    print(' procesando circle ',to_process)
    ids=to_process['ids']
    beach_st=status['beach']
    #elimina el elemento de la mitad en la secuencia especifica
    for i in range(len(beach_st) - 2 ):
        if beach_st[i:i + 3] == ids:
          k=i
          status['beach']=beach_st[:i+1]+beach_st[i+2:]
          print('status cambia a ',[ind_p[i] for i in status['beach']])
          break

    add_bisect(status['beach'][k],status['beach'][k+1])
    search_inter()
    events=[e for e in events if not ('ids' in e and ids[1] in e['ids'])]
    vor_vertices[(ids[0],ids[1],ids[2])]=to_process['coord']
